# ✨🦙 Getting started with Argilla's LlamaIndex Integration

This integration allows the user to include the feedback loop that Argilla offers into the LlamaIndex ecosystem. It's based on a callback handler to be run within the LlamaIndex workflow. 

Don't hesitate to check out both [LlamaIndex](https://github.com/run-llama/llama_index) and [Argilla](https://github.com/argilla-io/argilla)

## Getting started

You first need to install argilla and argilla-llama-index as follows:

In [ ]:
%pip install argilla-llama-index

You will need to an Argilla Server running to monitor the LLM. You can either install the server locally or have it on HuggingFace Spaces. For a complete guide on how to install and initialize the server, you can refer to the [Quickstart Guide](https://docs.argilla.io/en/latest/getting_started/quickstart_installation.html).

## Usage

It requires just a simple step to log your data into Argilla within your LlamaIndex workflow. We just need to call the handler before starting production with your LLM.

We will use GPT3.5 from OpenAI as our LLM. For this, you will need a valid API key from OpenAI. You can have more info and get one via [this link](https://openai.com/blog/openai-api).

After you get your API key, the easiest way to import it is through an environment variable, or via *getpass()*.

In [1]:
import os

openai_api_key = os.getenv("OPENAI_API_KEY")

Let's now write all the necessary imports

In [2]:
from llama_index.core import VectorStoreIndex, ServiceContext, SimpleDirectoryReader, set_global_handler
from llama_index.llms.openai import OpenAI

What we need to do is to set Argilla as the global handler as below. Within the handler, we need to provide the dataset name that we will use. If the dataset does not exist, it will be created with the given name. You can also set the API KEY, API URL, and the Workspace name. You can learn more about the variables that controls Argilla initialization [here](https://docs.argilla.io/en/latest/getting_started/installation/configurations/workspace_management.html).

In [3]:
import argilla as rg

rg.init(
    api_url="http://localhost:6900",
    api_key="owner.apikey",
    workspace="admin"
)

/Users/ignacio/Documents/recognai/argilla-llama-index/.venv/lib/python3.10/site-packages/argilla/client/client.py:195: UserWarning: You're connecting to Argilla Server 1.21.0 using a different client version (1.24.0).
This may lead to potential compatibility issues during your experience.
To ensure a seamless and optimized connection, we highly recommend aligning your client version with the server version.
  warnings.warn(


In [4]:
set_global_handler("argilla", dataset_name="query_model")

Let's now create the llm instance, using GPT-3.5 from OpenAI.

In [5]:
llm = OpenAI(model="gpt-3.5-turbo", temperature=0.8, openai_api_key=openai_api_key)

With the code snippet below, you can create a basic workflow with LlamaIndex. You will also need a txt file as the data source within a folder named "data". For a sample data file and more info regarding the use of Llama Index, you can refer to the [Llama Index documentation](https://docs.llamaindex.ai/en/stable/getting_started/starter_example.html).

In [10]:
service_context = ServiceContext.from_defaults(llm=llm)
docs = SimpleDirectoryReader("../data").load_data()
index = VectorStoreIndex.from_documents(docs, service_context=service_context)
query_engine = index.as_query_engine()

/var/folders/s0/4h4_2s7n3wxbm06wx8y43mm80000gn/T/ipykernel_86259/2413633764.py:1: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(llm=llm)


Now, let's run the `query_engine` to have a response from the model.

In [12]:
response = query_engine.query("What did the author do growing up?")
response

Response(response='The author worked on writing and programming before college.', source_nodes=[NodeWithScore(node=TextNode(id_='c9202a57-b440-4d86-9a5e-795ada63ba29', embedding=None, metadata={'file_path': '../data/paul_graham_essay.txt', 'file_name': 'paul_graham_essay.txt', 'file_type': 'text/plain', 'file_size': 75041, 'creation_date': '2024-02-21', 'last_modified_date': '2024-02-21', 'last_accessed_date': '2024-02-21'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='5ea14634-052e-4b7f-b930-9025ae6202b3', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'file_path': '../data/paul_graham_essay.txt', 'file_name': 'paul_graham_essay.txt', 'file_type': 'text/plain', 'file_size': 75041, 'creation_date': '202

The prompt given and the response obtained will be logged in to Argilla server. You can check the data on Argilla's UI:

![Argilla Dataset](../assets/argilla-ui-dataset.png)
